In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta import *

In [2]:
builder = SparkSession.builder \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/12/17 09:53:40 WARN Utils: Your hostname, phuc-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp45s0)
23/12/17 09:53:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/phuc/Practice/DataScience/DSProject/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/phuc/.ivy2/cache
The jars for the packages stored in: /home/phuc/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a18e5fd4-bd4d-4e16-91f3-95817526c38a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 116ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

### Read raw data

In [3]:
job_path = "/home/phuc/Practice/DataScience/DSProject/data/raw/careerlink/2023-12-17T02-43-28+00-00.csv"
job_df = spark.read.csv(job_path, header=True)

In [4]:
job_df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_listed: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- job_deadline: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- job_address: string (nullable = true)
 |-- job_experience_requied: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_function: string (nullable = true)
 |-- education_level: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Industries: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- skill: string (nullable = true)



### Transform and ingest into delta lake

In [7]:
from job_transformation.careerlink.transformation_and_ingestion import transform_and_ingest

daily_table = "/home/phuc/Practice/DataScience/DSProject/data/raw/careerlink/2023-12-17T02-43-28+00-00.csv"
ingest_table = "/home/phuc/Practice/DataScience/DSProject/data/ingestion/careerlink"

transform_and_ingest(
    daily_table=daily_table,
    ingest_table=ingest_table
)

In [6]:
ingest_df = spark.read.format("delta").load("/home/phuc/Practice/DataScience/DSProject/data/ingestion/careerlink")
ingest_df.show(truncate=True)

+-------+--------------------+----------+--------------------+------------+--------------------+-----------------+----------------------+---------------+--------------------+--------+--------------------+--------------------+-------------+--------------------+-----------+-----------+----------+----------+-----------+----------+
|post_id|           job_title|job_listed|              salary|job_deadline|        company_name|      job_address|job_experience_requied|employment_type|     education_level|  gender|     job_description|               skill|    job_level|            industry|job_yoe_min|job_yoe_max|salary_min|salary_max|ingested_at|updated_at|
+-------+--------------------+----------+--------------------+------------+--------------------+-----------------+----------------------+---------------+--------------------+--------+--------------------+--------------------+-------------+--------------------+-----------+-----------+----------+----------+-----------+----------+
|2733584| 

In [ ]:
ingest_df.printSchema()

root
 |-- Industries: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- education_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- job_address: string (nullable = true)
 |-- job_deadline: date (nullable = true)
 |-- job_description: string (nullable = true)
 |-- job_experience_requied: string (nullable = true)
 |-- job_listed: date (nullable = true)
 |-- job_title: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- skill: string (nullable = true)
 |-- job_role: string (nullable = true)
 |-- job_yoe_min: integer (nullable = true)
 |-- job_yoe_max: integer (nullable = true)
 |-- salary_min: integer (nullable = true)
 |-- salary_max: integer (nullable = true)
 |-- ingested_at: date (nullable = true)
 |-- updated_at: date (nullable = true)
 |-- job_level: string (nullable = true)

